In [1]:
import pandas as pd
import cobra
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import linprog
import scipy
import mip
from copy import deepcopy

def save_dict(data, name):
    with open(name, 'w' ) as file:
        json.dump( data, file )

# Community modeling

In this notebook we will implement a method to create community models of two or more species specific metabolic models using cobrapy.

In [45]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP_SNM.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854_SNM.xml")
print("Growth: ", model_DP.slim_optimize())
print("Growth: ", model_SA.slim_optimize())

Growth:  0.282365392532252
Growth:  2.558694612613397


In [3]:
for rec in model_SA.reactions:
    rec.lower_bound = max(rec.lower_bound, -1000)
    rec.upper_bound = min(rec.upper_bound, 1000)

In [4]:
snm3 = pd.read_csv("SNM3.csv", sep =";")
snm3.head()

,Compound,BiGG,ModelSeed,KEGG
0,Alanine,ala__L,cpd00035,C00041
1,Arginine,arg__L,cpd00051,C00062
2,Cysteine,cys__L,cpd00084,C00097
3,Glutamic acid,glu__L,cpd00023,C00025
4,Glycine,gly,cpd00033,C00037


In [5]:
BIOMASS_DP = "Growth" 
BIOMASS_SA = "BIOMASS_iYS_wild_type"
models = [model_DP.copy(), model_SA.copy()]

The following metabolites have been previously added to obtain growth on the medium: EX_ile__L_e, EX_met__L_e and EX_26dap__M_e for DP. In the community this may is not necessary as the metabolites could be produced from e.g. SA.

**PROBLEM:** SA does not have EX_26dap__M_e and hence cannot produce it for DP....

Thus we cannot drop EX_26dap__M_e from the medium otherwise DP could not grow...

In [6]:
print(model_SA.exchanges.EX_met__L_e)
print(model_SA.exchanges.EX_ile__L_e)
#print(model_SA.exchanges.EX_26dap__M_e)

EX_met__L_e: met__L_e --> 
EX_ile__L_e: ile__L_e --> 


In [7]:
# Can produces met
with model_SA as model: 
    model.objective = model.exchanges.EX_met__L_e
    print(model.slim_optimize())

10.0


In [8]:
# Can produces ile
with model_SA as model: 
    model.objective = model.exchanges.EX_ile__L_e
    print(model.slim_optimize())

21.406249999999982


In [14]:
from cobra.medium import minimal_medium

min_medium_DP = minimal_medium(model_DP, 0.1*model_DP.slim_optimize())

In [13]:
min_medium_SA = minimal_medium(model_SA, 0.1*model_SA.slim_optimize())

## 1) Constructing of community model explicitely

Here we introduce only shuttle reactions for reactions that are common in the uptake/sekretion reactions of the individual models!

In [15]:
def create_stoichiometry_matrix(model):
    metabolites = model.metabolites 
    reactions = model.reactions 
    S = np.zeros((len(metabolites), len(reactions)))
    
    met_id = dict()
    rec_id = dict()
    for i,reaction in enumerate(model.reactions):
        rec_id[reaction.id] = i
        for metabolite, stoich in reaction.metabolites.items():
            met_id[metabolite.id] = int(metabolites.index(metabolite))
            S[metabolites.index(metabolite), i] = stoich
    return S, met_id, rec_id 

In [16]:
class Model():
    def __init__(self, model, biomass_function):
        """ This is a new class of metabolic model, capable of flux balance analysis
        Attributes:
        models (list): CobraPy models of single organisms which will be used in construction
        biomass_reactions (list): List of strings containing the ids for the growth reactions
        """
        self.biomass_function = biomass_function
        self.model = model
        self.id = model.id
        # Compute stoichimetry_matrix
        S, met_id, rec_id = create_stoichiometry_matrix(model)
        self.num_reactions = S.shape[1]
        self.num_metabolites = S.shape[0]
        self.stoichiometry_matrix = scipy.sparse.csr_matrix(S)
        self.met_id = met_id
        self.rec_id = rec_id 
        # Set objective
        idx = self.rec_id[biomass_function]
        c = np.zeros(self.num_reactions)
        c[idx] = 1
        self.objective_c = c
        # Set bounds
        self._reset_bounds()
    
    @property
    def reactions(self):
        return self.model.reactions
    @property
    def exchanges(self):
        return self.model.exchanges
    @property
    def metabolites(self):
        return self.model.metabolites
    @property
    def medium(self):
        return self.model.medium

    def set_medium(self, medium):
        ex_ids = [ex.id for ex in self.exchanges]
        new_med = {}
        for key,val in medium.items():
            if key in ex_ids:
                new_med[key] = val
        self.model.medium = new_med
        self._reset_bounds()
        
    def optimize(self, disp=False):
        sol = linprog(-self.objective_c, A_eq=self.stoichiometry_matrix, b_eq=np.zeros(self.num_metabolites), bounds=self.bounds, method="highs", options={"disp":disp})
        sol["fun"] = -sol["fun"] # As we have to minimize
        return sol 
    
    def slim_optimize(self, disp=False):
        sol = self.optimize(disp=disp)
        return sol["fun"]

    def summary(self):
        sol = self.optimize()
        flux = sol["x"]
        ex_ids = [ex.id for ex in self.exchanges]
        fluxes = []
        for ex in ex_ids:
            idx = self.rec_id[ex]
            fluxes.append(flux[idx])
        summary_df = pd.DataFrame({"Exchange reaction": ex_ids, "Flux": fluxes})
        summary_df.sort_values(["Flux"], inplace=True)
        return summary_df

    def _reset_bounds(self):
        self.bounds = []
        for rec in self.model.reactions:
            self.bounds.append((rec.lower_bound, rec.upper_bound))

    def __add__(self, model2):
        """ Adding another model creates a community model """
        return CommunityModel([self,model2], [1.,1.])
    

In [17]:
model1 = Model(model_DP, BIOMASS_DP)
model2 = Model(model_SA, BIOMASS_SA)

$$ \max \sum_{i=1}^n \theta_i \text{ subject to }$$

$$ SV = 0, V_{j,min} \leq V_j \leq V_{j, max}, V_{Growth} \geq MBR/10 , V_{i, COOPM} + V_{i, min}\theta_i \geq V_{i,\min} $$

In [18]:
from mip import Model, xsum, maximize, BINARY

S1 = model1.stoichiometry_matrix.todense()
S1_dict = model1.rec_id 
bounds1 = model1.bounds 
obj1 = np.where(model1.objective_c > 0)[0][0]

S2 = model2.stoichiometry_matrix.todense()
S2_dict = model2.rec_id 
bounds2 = model2.bounds
obj2 = np.where(model2.objective_c > 0)[0][0]

In [19]:
snm3_med = dict(list(model_DP.medium.items()) + list(model_SA.medium.items()))

We will add following constraints:
* All EX_ile__L_e used by DP must be produced by SA.
* All EX_met__L_E used by DP must be produced by SA.

In [47]:
from mip import Model, xsum, maximize, BINARY

comm_model = Model()

# Shuttel reactions
x_sh = []
id1 = []
id2 = []
x_sh_dict = {}
for key, val in snm3_med.items():
    x = comm_model.add_var(lb=-val, ub=1000)
    x_sh +=[x]
    x_sh_dict[key] = x
    if key in S1_dict:
        id1 += [S1_dict[key]]
    else:
        id1 += [None]
    if key in S2_dict:
        id2 += [S2_dict[key]]
    else:
        id2 += [None]

# Flux first model
x1 = []
for i, (lb, ub) in enumerate(bounds1): 
    x1 += [comm_model.add_var(lb = lb, ub=ub)]

# Flux second model
x2 = []
for i, (lb, ub) in enumerate(bounds2): 
    x2 += [comm_model.add_var(lb = lb, ub=ub)]
# Stoichiometry
for i in range(S1.shape[0]):
    comm_model.add_constr(xsum(S1[i,j]*x1[j] for j in range(S1.shape[1])) == 0)

for i in range(S2.shape[0]):
    comm_model.add_constr(xsum(S2[i,j]*x2[j] for j in range(S2.shape[1])) == 0)

ids_ile = (0,0)
ids_met = (0,0)
# Shuttel constraints
for i,key in enumerate(snm3_med):
    # if "EX_ile__L_e" == key:
    #     idx1 = id1[i]
    #     idx2 = id2[i]
    #     ids_ile = (idx1, idx2)
        
    #     # Sa must produce it...
    #     comm_model.add_constr(x1[idx1] + x2[idx2] == 0)
    # elif "EX_met__L_e" == key:
    #     idx1 = id1[i]
    #     idx2 = id2[i]
    #     ids_met = (idx1, idx2)
    #     # Sa must produce it
    #     comm_model.add_constr(x1[idx1] + x2[idx2] == 0)
    if id1[i] is not None and id2[i] is not None:
        idx1 = id1[i]
        idx2 = id2[i]
        comm_model.add_constr(-x_sh[i] + x1[idx1] + x2[idx2] == 0)
    elif id1[i] is not None:
        idx = id1[i]
        comm_model.add_constr(-x_sh[i] + x1[idx] == 0)
    else:
        idx = id2[i]
        comm_model.add_constr(-x_sh[i] + x2[idx] == 0)



In [48]:
def get_exchange_flux(S_dict, x):
    dic ={}
    for key, val in S_dict.items():
        if "EX_" in key:
            dic[key] = x[val].x
    return dic

def get_medium(x):
    dic = {}
    for key, val in x.items():
        dic[key] = val.lb
    return dic

First looking at the convex combinaiton

In [49]:
comm_model.objective = maximize(x1[obj1] + x2[obj2])
comm_model.optimize()
print(x2[obj2].x)
print(x1[obj1].x)

2.5586946126135386
0.0


In [52]:
# SA dominates
w1 = 0.85
w2 = 0.15

comm_model.objective = maximize(w1*x1[obj1] + w2*x2[obj2])
comm_model.optimize()
print(x1[obj1].x)
print(x2[obj2].x)
minMBR = (w1*x1[obj1].x + w2*x2[obj2].x)/10
print(minMBR)

0.23934339136846774
1.723758259485812
0.04620056215860694


In [184]:
shared_keys = set(min_medium_DP.keys()).intersection(set(min_medium_SA.keys()))
shared_medium = dict()
for key in shared_keys:
    shared_medium[key] = min(min_medium_DP[key], min_medium_SA[key])

In [185]:
shared_medium

{'EX_his__L_e': 0.002590591762669045,
 'EX_fe2_e': 0.0021067642024968536,
 'EX_ribflv_e': 1.2195781272038601e-05,
 'EX_k_e': 0.0015045698519940388,
 'EX_trp__L_e': 0.0015543605265616381,
 'EX_cl_e': 0.0012670655721661631,
 'EX_glc__D_e': 0.9999999999999942,
 'EX_o2_e': 0.012577861281038893,
 'EX_mn2_e': 0.00016836210550996272,
 'EX_arg__L_e': 0.07486251744579463,
 'EX_nac_e': 2.162455164908833e-05,
 'EX_leu__L_e': 0.012319680565634147,
 'EX_mg2_e': 0.0021116906637898515,
 'EX_phe__L_e': 0.008836773254827965,
 'EX_so4_e': 0.0006123728011132153,
 'EX_lys__L_e': 0.009383669275995604,
 'EX_zn2_e': 8.290170544867465e-05,
 'EX_na1_e': 0.0025587956488567903,
 'EX_thm_e': 6.0978906360193005e-06,
 'EX_cys__L_e': 0.0026543116397897693}

In [186]:
for id, x in x_sh_dict.items():
    if id in shared_medium:
        x.lb = -min_medium_DP[id]
    else:
        x.lb = 0

In [187]:
comm_model.objective = maximize(w1*x1[obj1] + w2*x2[obj2])
comm_model.optimize()
print(x1[obj1].x)
print(x2[obj2].x)
minMBR = (w1*x1[obj1].x + w2*x2[obj2].x)/10

0.0
0.001286150112519139


In [188]:
interchange = {"DP_flux":[], "SA_flux":[]}
index = list()
for key, val in get_exchange_flux(S1_dict,x1).items():
    other_echange = get_exchange_flux(S2_dict,x2)
    if val < 0:
        print(key, val)
        index += [key]
        if key in other_echange:
            print("Other exchange",key, other_echange[key])

EX_glc__D_e -0.868651696924108
Other exchange EX_glc__D_e -0.1313483030758862
EX_arg__L_e -0.03287324118981504
Other exchange EX_arg__L_e -0.0003861381933806529
EX_o2_e -0.012577861281038894
Other exchange EX_o2_e 0.0
EX_glu__L_e -0.031813319336022945
Other exchange EX_glu__L_e 0.031813319336022945
EX_ile__L_e -0.025155722562124984
Other exchange EX_ile__L_e 0.02515572256212499


This is expected as the medium does not contain 26dap and SA cannot produce it, thus we will add it to the medium.

In [189]:
min_medium_DP

EX_glc__D_e      1.000000
EX_leu__L_e      0.012320
EX_4abz_e        0.000018
EX_cl_e          0.001659
EX_pi_e          0.024136
EX_ribflv_e      0.000012
EX_gly_e         0.028193
EX_thr__L_e      0.006937
EX_arg__L_e      0.074863
EX_lys__L_e      0.009384
EX_k_e           0.001505
EX_pro__L_e      0.006045
EX_ca2_e         0.001468
EX_mg2_e         0.002420
EX_mn2_e         0.001071
EX_cobalt2_e     0.000998
EX_zn2_e         0.000899
EX_cu2_e         0.000926
EX_o2_e          0.012578
EX_glu__L_e      0.017145
EX_fe2_e         0.002107
EX_his__L_e      0.002591
EX_ile__L_e      0.007944
EX_met__L_e      0.004209
EX_so4_e         0.000612
EX_trp__L_e      0.001554
EX_val__L_e      0.011721
EX_thm_e         0.000006
EX_cys__L_e      0.002654
EX_phe__L_e      0.008837
EX_na1_e         0.002559
EX_ni2_e         0.001002
EX_nac_e         0.000336
EX_26dap__M_e    0.002734
dtype: float64

In [190]:
def set_medium(medium):
    for id, x in x_sh_dict.items():
        if id in medium:
            x.lb = -min_medium_DP[id]
        else:
            x.lb = 0

In [191]:
set_medium(shared_medium)

In [192]:
comm_model.objective = maximize(w1*x1[obj1] + w2*x2[obj2])
comm_model.optimize()
print(x1[obj1].x)
print(x2[obj2].x)
minMBR = (w1*x1[obj1].x + w2*x2[obj2].x)/10

0.0
0.001286150112519139


In [193]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [203]:
missing = set(min_medium_DP.keys()).difference(set(shared_keys))
missing_combinations = powerset(missing)

In [204]:
best_medium=None
for combi in missing_combinations:
    medium = shared_medium.copy()
    medium["EX_26dap__M_e"] = 0.002734
    for ex in combi:
        medium[ex] = min_medium_DP[ex]
    set_medium(medium)
    comm_model.objective = maximize(x1[obj1])
    comm_model.optimize()
    if x1[obj1].x > 0:
        if best_medium is None:
            print(combi, x1[obj1].x)
            best_medium = medium
        elif len(medium) < len(best_medium):
            print(combi, x1[obj1].x)
            best_medium = medium 
        else:
            pass

('EX_thr__L_e', 'EX_cu2_e', 'EX_ni2_e', 'EX_pro__L_e', 'EX_ca2_e', 'EX_cobalt2_e') 0.00709836307723159


As expected these metabolits are **no exchange reactions for SA**, hence they cannot be produced by it. Thus we will have to substitute them into the medium

Hence we will add the following increasients to the medium: 'EX_thr__L_e', 'EX_cu2_e', 'EX_ni2_e', 'EX_pro__L_e', 'EX_ca2_e', 'EX_cobalt2_e'

In [205]:
best_medium

{'EX_his__L_e': 0.002590591762669045,
 'EX_fe2_e': 0.0021067642024968536,
 'EX_ribflv_e': 1.2195781272038601e-05,
 'EX_k_e': 0.0015045698519940388,
 'EX_trp__L_e': 0.0015543605265616381,
 'EX_cl_e': 0.0012670655721661631,
 'EX_glc__D_e': 0.9999999999999942,
 'EX_o2_e': 0.012577861281038893,
 'EX_mn2_e': 0.00016836210550996272,
 'EX_arg__L_e': 0.07486251744579463,
 'EX_nac_e': 2.162455164908833e-05,
 'EX_leu__L_e': 0.012319680565634147,
 'EX_mg2_e': 0.0021116906637898515,
 'EX_phe__L_e': 0.008836773254827965,
 'EX_so4_e': 0.0006123728011132153,
 'EX_lys__L_e': 0.009383669275995604,
 'EX_zn2_e': 8.290170544867465e-05,
 'EX_na1_e': 0.0025587956488567903,
 'EX_thm_e': 6.0978906360193005e-06,
 'EX_cys__L_e': 0.0026543116397897693,
 'EX_26dap__M_e': 0.002734,
 'EX_thr__L_e': 0.0069370205460978955,
 'EX_cu2_e': 0.0009257250408242616,
 'EX_ni2_e': 0.0010022613010017905,
 'EX_pro__L_e': 0.006044704997960757,
 'EX_ca2_e': 0.0014677908938624288,
 'EX_cobalt2_e': 0.0009981830860061644}

In [206]:
set_medium(best_medium)

In [207]:
comm_model.objective = maximize(w1*x1[obj1] + w2*x2[obj2])
comm_model.optimize()
print(x1[obj1].x)
print(x2[obj2].x)
minMBR = (w1*x1[obj1].x + w2*x2[obj2].x)/10

0.007098367882456253
0.0009628255517842001


In [208]:
interchange = {"DP_flux":[], "SA_flux":[]}
index = list()
for key, val in get_exchange_flux(S2_dict,x2).items():
    other_echange = get_exchange_flux(S1_dict,x1)
    if val < 0:
        print(key, val)
        index += [key]
        if key in other_echange:
            print("Other exchange",key, other_echange[key])

EX_h2o_e -0.029945258591327507
Other exchange EX_h2o_e 0.4904757172624663
EX_arg__L_e -0.025080738842006508
Other exchange EX_arg__L_e -0.049781778603788115
EX_his__L_e -5.800741764403502e-05
Other exchange EX_his__L_e -0.0006512474209312142
EX_k_e -0.00017878707671080808
Other exchange EX_k_e -0.0003782329774385043
EX_leu__L_e -0.0003023377080732263
Other exchange EX_leu__L_e -0.0030970376385353383
EX_lys__L_e -0.0003470738841987999
Other exchange EX_lys__L_e -0.0023589553950281437
EX_cl_e -4.767912132435359e-06
Other exchange EX_cl_e -0.00041712313264840407
EX_cys__L_e -0.001218179949249762
Other exchange EX_cys__L_e -0.0014361316905400073
EX_mg2_e -7.946199278875004e-06
Other exchange EX_mg2_e -0.0006084454401063097
EX_mn2_e -6.335392130740038e-07
Other exchange EX_mn2_e -0.0002691807716321314
EX_mobd_e -6.7397788630923146e-09
EX_na1_e -0.0019155414352746666
Other exchange EX_na1_e -0.0006432542135821236
EX_fe3_e -1.4209379493231231e-05
EX_nac_e -8.137231410366602e-08
Other exchange